In [2]:
import pandas as pd
import numpy as np
import os

In [ ]:
pd.set_option('display.max_columns', None)
current_dir = os.getcwd()

## DATA

In [25]:
url_sales = os.path.join(current_dir, "../data/raw/processed_data.csv")
sales = pd.read_csv(url_sales, sep=",")

/var/folders/js/4g9f6f7d2vj886wxt_jkqkyc0000gn/T/ipykernel_3075/792020892.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sales = pd.read_csv(url_sales, sep=",")


In [26]:
sales.head()

,CategoryID,CategoryName,ProductName,Price,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,SalesID,CustomerID,Quantity,ProductID,Discount,TotalPrice,SalesDate,TransactionNumber,FirstName,MiddleInitial,LastName,CityID,Address,CityName,Zipcode,CountryID,CountryName,CountryCode
0,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,338,95780,25,15,0.0,0.0,2018-02-22 10:29:00.960,8DHYWB12G95EZUP3WC9H,Cristina,U,Costa,18,632 Oak Avenue,Little Rock,81251,32,United States,AR
1,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1027,81481,21,15,0.2,0.0,2018-01-11 17:52:45.710,Q1JIS5574U15ZOYMNKIB,Scotty,B,Carney,10,34 Rocky Old St.,Toledo,52048,32,United States,AR
2,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1252,68697,18,15,0.0,0.0,2018-04-19 23:14:01.500,UIO1OARQ55ZWNZN2VPYX,Tamara,Q,Orr,69,76 South White Oak Avenue,Oakland,29029,32,United States,AR
3,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1681,33559,9,15,0.2,0.0,2018-04-06 00:16:27.280,FQYKYNAUU24F9PFGCJER,Ross,T,Bush,13,220 East White Milton Avenue,Akron,83448,32,United States,AR
4,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,2229,59905,16,15,0.0,0.0,2018-04-30 21:06:49.980,QEMGDAG28IZ0GDKWCROB,Ricky,C,Heath,14,90 White Old St.,Indianapolis,81678,32,United States,AR


In [27]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6758125 entries, 0 to 6758124
Data columns (total 27 columns):
 #   Column             Dtype  
---  ------             -----  
 0   CategoryID         int64  
 1   CategoryName       object 
 2   ProductName        object 
 3   Price              float64
 4   Class              object 
 5   ModifyDate         object 
 6   Resistant          object 
 7   IsAllergic         object 
 8   VitalityDays       float64
 9   SalesID            int64  
 10  CustomerID         int64  
 11  Quantity           int64  
 12  ProductID          int64  
 13  Discount           float64
 14  TotalPrice         float64
 15  SalesDate          object 
 16  TransactionNumber  object 
 17  FirstName          object 
 18  MiddleInitial      object 
 19  LastName           object 
 20  CityID             int64  
 21  Address            object 
 22  CityName           object 
 23  Zipcode            int64  
 24  CountryID          int64  
 25  CountryName       

In [28]:
sales.shape

(6758125, 27)

#### Modificar la columna fecha para quedarnos con solo la fecha y descartar la hora en la columa que nos interesa:

#### Sales date

In [30]:
sales['SalesDate'] = pd.to_datetime(sales['SalesDate']).dt.date

sales.rename(columns = {"SalesDate" : "Date" }, inplace = True)

sales['Date']

0          2018-02-22
1          2018-01-11
2          2018-04-19
3          2018-04-06
4          2018-04-30
              ...    
6758120    2018-03-06
6758121    2018-03-02
6758122    2018-03-04
6758123    2018-03-12
6758124    2018-01-13
Name: Date, Length: 6758125, dtype: object

#### Crear columna de estados segun ciudades

In [31]:
sales['CityName'].unique()

array(['Little Rock', 'Toledo', 'Oakland', 'Akron', 'Indianapolis',
       'Stockton', 'Baltimore', 'Dallas', 'Albuquerque', 'Anaheim',
       'Colorado', 'Honolulu', 'Buffalo', 'Lincoln', 'Omaha', 'Mobile',
       'Phoenix', 'Newark', 'Houston', 'San Francisco', 'San Jose',
       'Charlotte', 'Glendale', 'Grand Rapids', 'Denver', 'New York',
       'Cincinnati', 'Tulsa', 'Cleveland', 'Yonkers', 'Montgomery',
       'St. Paul', 'Riverside', 'Austin', 'St. Petersburg', 'Tacoma',
       'Seattle', 'Lubbock', 'Louisville', 'Spokane', 'Santa Ana',
       'Long Beach', 'Garland', 'Bakersfield', 'Oklahoma', 'Dayton',
       'Milwaukee', 'Hialeah', 'Fresno', 'Richmond', 'Shreveport',
       'Baton Rouge', 'Minneapolis', 'Fort Wayne', 'Des Moines',
       'Fort Worth', 'Kansas', 'Mesa', 'San Antonio', 'Nashville',
       'Portland', 'Atlanta', 'Aurora', 'Fremont', 'Washington',
       'Philadelphia', 'Greensboro', 'St. Louis', 'San Diego', 'Jackson',
       'Tucson', 'Pittsburgh', 'Boston', '

In [44]:
# Creamos un diccionario que mapea cada ciudad a su estado correspondiente

city_to_state = {
    'Little Rock': 'Arkansas',
    'Toledo': 'Ohio',
    'Oakland': 'California',
    'Akron': 'Ohio',
    'Indianapolis': 'Indiana',
    'Stockton': 'California',
    'Baltimore': 'Maryland',
    'Dallas': 'Texas',
    'Albuquerque': 'New Mexico',
    'Anaheim': 'California',
    'Honolulu': 'Hawaii',
    'Buffalo': 'New York',
    'Lincoln': 'Nebraska',
    'Omaha': 'Nebraska',
    'Mobile': 'Alabama',
    'Phoenix': 'Arizona',
    'Newark': 'New Jersey',
    'Houston': 'Texas',
    'San Francisco': 'California',
    'San Jose': 'California',
    'Charlotte': 'North Carolina',
    'Glendale': 'Arizona',
    'Grand Rapids': 'Michigan',
    'Denver': 'Colorado',
    'New York': 'New York',
    'Cincinnati': 'Ohio',
    'Tulsa': 'Oklahoma',
    'Cleveland': 'Ohio',
    'Yonkers': 'New York',
    'Montgomery': 'Alabama',
    'St. Paul': 'Minnesota',
    'Riverside': 'California',
    'Austin': 'Texas',
    'St. Petersburg': 'Florida',
    'Tacoma': 'Washington',
    'Seattle': 'Washington',
    'Lubbock': 'Texas',
    'Louisville': 'Kentucky',
    'Spokane': 'Washington',
    'Santa Ana': 'California',
    'Long Beach': 'California',
    'Garland': 'Texas',
    'Bakersfield': 'California',
    'Oklahoma': 'Oklahoma',
    'Dayton': 'Ohio',
    'Milwaukee': 'Wisconsin',
    'Hialeah': 'Florida',
    'Fresno': 'California',
    'Richmond': 'Virginia',
    'Shreveport': 'Louisiana',
    'Baton Rouge': 'Louisiana',
    'Minneapolis': 'Minnesota',
    'Fort Wayne': 'Indiana',
    'Des Moines': 'Iowa',
    'Fort Worth': 'Texas',
    'Kansas': 'Kansas',
    'Mesa': 'Arizona',
    'San Antonio': 'Texas',
    'Nashville': 'Tennessee',
    'Portland': 'Oregon',
    'Atlanta': 'Georgia',
    'Aurora': 'Colorado',
    'Colorado': 'Colorado',
    'Fremont': 'California',
    'Washington': 'District of Columbia',
    'Philadelphia': 'Pennsylvania',
    'Greensboro': 'North Carolina',
    'St. Louis': 'Missouri',
    'San Diego': 'California',
    'Jackson': 'Mississippi',
    'Tucson': 'Arizona',
    'Pittsburgh': 'Pennsylvania',
    'Boston': 'Massachusetts',
    'Columbus': 'Ohio',
    'Las Vegas': 'Nevada',
    'Rochester': 'New York',
    'El Paso': 'Texas',
    'Birmingham': 'Alabama',
    'Virginia Beach': 'Virginia',
    'Corpus Christi': 'Texas',
    'Los Angeles': 'California',
    'Memphis': 'Tennessee',
    'Raleigh': 'North Carolina',
    'Detroit': 'Michigan',
    'Wichita': 'Kansas',
    'Jacksonville': 'Florida',
    'Tampa': 'Florida',
    'Miami': 'Florida',
    'Jersey': 'New Jersey',
    'Arlington': 'Texas',
    'Chicago': 'Illinois',
    'New Orleans': 'Louisiana',
    'Sacramento': 'California',
    'Norfolk': 'Virginia',
    'Madison': 'Wisconsin',
    'Anchorage': 'Alaska'
}

# Aplicamos este diccionario para crear una nueva columna 'State' 

sales['State'] = sales['CityName'].map(city_to_state)

sales['State'].unique()


array(['Arkansas', 'Ohio', 'California', 'Indiana', 'Maryland', 'Texas',
       'New Mexico', 'Colorado', 'Hawaii', 'New York', 'Nebraska',
       'Alabama', 'Arizona', 'New Jersey', 'North Carolina', 'Michigan',
       'Oklahoma', 'Minnesota', 'Florida', 'Washington', 'Kentucky',
       'Wisconsin', 'Virginia', 'Louisiana', 'Iowa', 'Kansas',
       'Tennessee', 'Oregon', 'Georgia', 'District of Columbia',
       'Pennsylvania', 'Missouri', 'Mississippi', 'Massachusetts',
       'Nevada', 'Illinois', 'Alaska'], dtype=object)

In [45]:
sales.head()

,CategoryID,CategoryName,ProductName,Price,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,SalesID,CustomerID,Quantity,ProductID,Discount,TotalPrice,Date,TransactionNumber,FirstName,MiddleInitial,LastName,CityID,Address,CityName,Zipcode,CountryID,CountryName,CountryCode,State
0,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,338,95780,25,15,0.0,0.0,2018-02-22,8DHYWB12G95EZUP3WC9H,Cristina,U,Costa,18,632 Oak Avenue,Little Rock,81251,32,United States,AR,Arkansas
1,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1027,81481,21,15,0.2,0.0,2018-01-11,Q1JIS5574U15ZOYMNKIB,Scotty,B,Carney,10,34 Rocky Old St.,Toledo,52048,32,United States,AR,Ohio
2,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1252,68697,18,15,0.0,0.0,2018-04-19,UIO1OARQ55ZWNZN2VPYX,Tamara,Q,Orr,69,76 South White Oak Avenue,Oakland,29029,32,United States,AR,California
3,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1681,33559,9,15,0.2,0.0,2018-04-06,FQYKYNAUU24F9PFGCJER,Ross,T,Bush,13,220 East White Milton Avenue,Akron,83448,32,United States,AR,Ohio
4,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,2229,59905,16,15,0.0,0.0,2018-04-30,QEMGDAG28IZ0GDKWCROB,Ricky,C,Heath,14,90 White Old St.,Indianapolis,81678,32,United States,AR,Indiana


# Ingreso nacional bruto + ((((((SALES))))))

In [51]:
url_inb = os.path.join(current_dir, "../data/raw/macrodata/ingreso_nacional_bruto.csv")
inb = pd.read_csv(url_inb, sep=",")

inb.head()

,Estado,Poblacion_2017,Poblacion_2018,Crecimiento (%)
0,.Arizona,7044008.0,7158024.0,1.62
1,.Arkansas,3001345.0,3009733.0,0.28
2,.California,39358497.0,39461588.0,0.26
3,.Colorado,5611885.0,5691287.0,1.41
4,.Connecticut,3573297.0,3571520.0,-0.05


#### Eliminamos los puntos de la columna estado

In [52]:
inb['Estado'] = inb['Estado'].str.replace(".", "", regex=False)

inb.head()

,Estado,Poblacion_2017,Poblacion_2018,Crecimiento (%)
0,Arizona,7044008.0,7158024.0,1.62
1,Arkansas,3001345.0,3009733.0,0.28
2,California,39358497.0,39461588.0,0.26
3,Colorado,5611885.0,5691287.0,1.41
4,Connecticut,3573297.0,3571520.0,-0.05


#### Cambiamos el nombre de la columa estado

In [53]:
inb.rename(columns = {"Estado" : "State" }, inplace = True)

In [59]:
inb_final = inb[['State','Crecimiento (%)']]

inb_final.head()

,State,Crecimiento (%)
0,Arizona,1.62
1,Arkansas,0.28
2,California,0.26
3,Colorado,1.41
4,Connecticut,-0.05


# Pib por estado + sales

In [67]:
url_pib = os.path.join(current_dir, "../data/raw/macrodata/pib_por_estados.csv")
pib = pd.read_csv(url_pib, sep=",")

pib.head()

,cp,estado,january,february,march,april,may
0,0,United States,6.681359e+15,6.681359e+15,6.681359e+15,6.716825e+15,6.716825e+15
1,1000,Alabama,7.335403e+13,7.335403e+13,7.335403e+13,7.339050e+13,7.339050e+13
2,2000,Alaska,1.763033e+13,1.763033e+13,1.763033e+13,1.748017e+13,1.748017e+13
3,4000,Arizona,1.145728e+14,1.145728e+14,1.145728e+14,1.149027e+14,1.149027e+14
4,5000,Arkansas,4.205223e+13,4.205223e+13,4.205223e+13,4.202373e+13,4.202373e+13


In [68]:
pib.rename(columns = {"estado" : "State" }, inplace = True)

In [ ]:
# Paso 1: Asegurarse de que la fecha esté en formato datetime
sales['Date'] = pd.to_datetime(sales['Date'])

# Paso 2: Extraer el nombre del mes en minúscula
sales['mes_nombre'] = sales['Date'].dt.month_name().str.lower()

# Paso 3: Crear una función que devuelva el valor correcto del PIB
def obtener_pib(row):
    estado = row['State']
    mes = row['mes_nombre']
    
    # Filtrar el DataFrame pib para ese estado
    fila_pib = pib[pib['State'] == estado]
    
    # Si encontramos el estado y el mes, devolvemos el valor
    if not fila_pib.empty and mes in fila_pib.columns:
        return fila_pib.iloc[0][mes]
    else:
        return None  

# Paso 4: Aplicar la función a cada fila del DataFrame
sales['pib'] = sales.apply(obtener_pib, axis=1)

# (Opcional) Eliminar columna auxiliar
sales.drop(columns='mes_nombre', inplace=True)

sales.head()

,CategoryID,CategoryName,ProductName,Price,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,SalesID,CustomerID,Quantity,ProductID,Discount,TotalPrice,Date,TransactionNumber,FirstName,MiddleInitial,LastName,CityID,Address,CityName,Zipcode,CountryID,CountryName,CountryCode,State,month,pib
0,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,338,95780,25,15,0.0,0.0,2018-02-22,8DHYWB12G95EZUP3WC9H,Cristina,U,Costa,18,632 Oak Avenue,Little Rock,81251,32,United States,AR,arkansas,2.0,4.205223e+13
1,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1027,81481,21,15,0.2,0.0,2018-01-11,Q1JIS5574U15ZOYMNKIB,Scotty,B,Carney,10,34 Rocky Old St.,Toledo,52048,32,United States,AR,ohio,1.0,2.175128e+14
2,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1252,68697,18,15,0.0,0.0,2018-04-19,UIO1OARQ55ZWNZN2VPYX,Tamara,Q,Orr,69,76 South White Oak Avenue,Oakland,29029,32,United States,AR,california,4.0,9.460646e+14
3,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1681,33559,9,15,0.2,0.0,2018-04-06,FQYKYNAUU24F9PFGCJER,Ross,T,Bush,13,220 East White Milton Avenue,Akron,83448,32,United States,AR,ohio,4.0,2.176970e+14
4,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,2229,59905,16,15,0.0,0.0,2018-04-30,QEMGDAG28IZ0GDKWCROB,Ricky,C,Heath,14,90 White Old St.,Indianapolis,81678,32,United States,AR,indiana,4.0,1.225487e+14


# Riqueza per capita + sales

In [91]:
url_rpc = os.path.join(current_dir, "../data/raw/macrodata/riqueza_per_capita.csv")
rpc = pd.read_csv(url_rpc, sep=",")

rpc.head()

,cp,estado,january,february,march,april,may
0,0,UnitedStates,1.748100e+10,1.748100e+10,1.748100e+10,1.766200e+10,1.766200e+10
1,1000,Alabama,1.360833e+10,1.360833e+10,1.360833e+10,1.369800e+10,1.369800e+10
2,2000,Alaska,1.935867e+10,1.935867e+10,1.935867e+10,1.949667e+10,1.949667e+10
3,4000,Arizona,1.487600e+10,1.487600e+10,1.487600e+10,1.498700e+10,1.498700e+10
4,5000,Arkansas,1.414467e+10,1.414467e+10,1.414467e+10,1.425400e+10,1.425400e+10


In [93]:
rpc.rename(columns = {"estado" : "State" }, inplace = True)

In [94]:
# Paso 1: Asegurarse de que la fecha esté en formato datetime
sales['Date'] = pd.to_datetime(sales['Date'])

# Paso 2: Extraer el nombre del mes en minúscula
sales['mes_nombre'] = sales['Date'].dt.month_name().str.lower()

# Paso 3: Crear una función que devuelva el valor correcto del PIB
def obtener_rpc(row):
    estado = row['State']
    mes = row['mes_nombre']
    
    # Filtrar el DataFrame pib para ese estado
    fila_rpc = rpc[pib['State'] == estado]
    
    # Si encontramos el estado y el mes, devolvemos el valor
    if not fila_rpc.empty and mes in fila_rpc.columns:
        return fila_rpc.iloc[0][mes]
    else:
        return None  

# Paso 4: Aplicar la función a cada fila del DataFrame
sales['rpc'] = sales.apply(obtener_rpc, axis=1)

# (Opcional) Eliminar columna auxiliar
sales.drop(columns='mes_nombre', inplace=True)

sales.head()

,CategoryID,CategoryName,ProductName,Price,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,SalesID,CustomerID,Quantity,ProductID,Discount,TotalPrice,Date,TransactionNumber,FirstName,MiddleInitial,LastName,CityID,Address,CityName,Zipcode,CountryID,CountryName,CountryCode,State,month,pib,rpc
0,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,338,95780,25,15,0.0,0.0,2018-02-22,8DHYWB12G95EZUP3WC9H,Cristina,U,Costa,18,632 Oak Avenue,Little Rock,81251,32,United States,AR,arkansas,2.0,4.205223e+13,1.414467e+10
1,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1027,81481,21,15,0.2,0.0,2018-01-11,Q1JIS5574U15ZOYMNKIB,Scotty,B,Carney,10,34 Rocky Old St.,Toledo,52048,32,United States,AR,ohio,1.0,2.175128e+14,1.570400e+10
2,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1252,68697,18,15,0.0,0.0,2018-04-19,UIO1OARQ55ZWNZN2VPYX,Tamara,Q,Orr,69,76 South White Oak Avenue,Oakland,29029,32,United States,AR,california,4.0,9.460646e+14,2.020400e+10
3,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,1681,33559,9,15,0.2,0.0,2018-04-06,FQYKYNAUU24F9PFGCJER,Ross,T,Bush,13,220 East White Milton Avenue,Akron,83448,32,United States,AR,ohio,4.0,2.176970e+14,1.584300e+10
4,1,Confections,"Spoon - Soup, Plastic",32.442,Low,2017-03-03 09:47:09.310,Weak,True,0.0,2229,59905,16,15,0.0,0.0,2018-04-30,QEMGDAG28IZ0GDKWCROB,Ricky,C,Heath,14,90 White Old St.,Indianapolis,81678,32,United States,AR,indiana,4.0,1.225487e+14,1.542933e+10


# Wti Crude Oil + (((((SALES)))))

In [96]:
url_wti_crude_oil = os.path.join(current_dir, "../data/raw/macrodata/wti_crude_oil.csv")
wti_oil_raw = pd.read_csv(url_wti_crude_oil, sep = ",")
wti_oil_raw

,Date,Open,High,Low,Close,Adj Close,Volume
0,"May 8, 2018",7.02,7.24,6.59,7.22,7.0267,"5,904,900"
1,"May 7, 2018",6.80,7.35,6.79,6.96,6.7737,"5,220,500"
2,"May 4, 2018",6.49,6.80,6.36,6.64,6.4622,"3,152,800"
3,"May 3, 2018",6.60,6.73,6.07,6.38,6.2092,"3,944,300"
4,"May 2, 2018",6.30,6.75,6.17,6.74,6.5595,"4,852,500"
...,...,...,...,...,...,...,...
83,"Jan 8, 2018",4.40,4.47,4.24,4.46,4.3406,"2,576,100"
84,"Jan 5, 2018",4.08,4.39,4.03,4.36,4.2433,"3,298,400"
85,"Jan 4, 2018",4.18,4.29,3.91,4.24,4.1265,"3,950,500"
86,"Jan 3, 2018",3.83,4.30,3.82,4.16,4.0486,"5,485,800"


In [97]:
wti_oil = wti_oil_raw[["Date", "Close"]]
wti_oil

,Date,Close
0,"May 8, 2018",7.22
1,"May 7, 2018",6.96
2,"May 4, 2018",6.64
3,"May 3, 2018",6.38
4,"May 2, 2018",6.74
...,...,...
83,"Jan 8, 2018",4.46
84,"Jan 5, 2018",4.36
85,"Jan 4, 2018",4.24
86,"Jan 3, 2018",4.16


# Tasa de desempleo + (((((((SALES)))))))

In [98]:
url_td = os.path.join(current_dir, "../data/raw/macrodata/tasa_desempleo_por_estados.csv")
td = pd.read_csv(url_td, sep=",")

td.head()

,Place,Year,Month,Unemployment Rate
0,Alabama,2018,May,3.5
1,Alabama,2018,April,3.5
2,Alabama,2018,March,4.0
3,Alabama,2018,February,4.4
4,Alabama,2018,January,4.4


# population_density

# personal_income

# population_growth

In [ ]:
# prueba switch branch